<a href="https://colab.research.google.com/github/660710078/TopGun2025/blob/main/poc_tesa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn

In [2]:
import uvicorn
import gradio as gr #แดชวบอร์ด
import requests
import datetime

In [3]:
# ใช้ Fast API
#สร้างเส้น get ที่ /predict
# ได้ไฟล์ main มา
with open('main.py', 'w') as f:
    f.write('''import io
from fastapi import FastAPI
from fastapi.responses import JSONResponse
import random

app = FastAPI()

@app.get("/predict")
async def predict():
    random_integer = random.randint(1, 100)
    return JSONResponse(content={"result": random_integer})
''')

In [4]:
!ls

main.py  sample_data


In [5]:
# !kill -9 $(lsof -t -i:8000)
#รันบนlocal 8000

In [6]:
!nohup uvicorn main:app --host 0.0.0.0 --port 8000 --reload &
# webserver ของ python unicorn linux

nohup: appending output to 'nohup.out'


In [7]:
!cat nohup.out

In [8]:
!curl -X GET http://0.0.0.0:8000/predict

curl: (7) Failed to connect to 0.0.0.0 port 8000 after 1 ms: Connection refused


In [9]:
# สร้าง dashboard
API_URL = "http://0.0.0.0:8000/predict"

def get_data():
    res = requests.get(API_URL)
    data = res.json()
    thb_rate = data["result"]
    return f"1 USD = {thb_rate:.2f} THB"

# UI
with gr.Blocks() as demo:
    gr.Markdown("### อัปเดตราคาค่าเงินจาก API (ไม่ reload หน้า)")
    output = gr.Textbox(label="ผลลัพธ์", interactive=False)
    btn = gr.Button("รีเฟรชข้อมูล")

    btn.click(fn=get_data, outputs=output)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5c354fee1f020e667c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
import gradio as gr
import requests, datetime

def get_data():
    r = requests.get("http://0.0.0.0:8000/predict")
    data = r.json()
    return f'{datetime.datetime.now():%H:%M:%S} → BTC/USD: {data["result"]}'

with gr.Blocks() as app:
    out = gr.Textbox(label="Realtime Feed", lines=2)

    # Gradio 4.x+ syntax
    gr.Timer(value=1).tick(fn=get_data, outputs=out)

app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7905fc47a9589c407e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
import gradio as gr
import requests
import datetime
import plotly.graph_objects as go
from collections import deque

# Keep track of data for the last few updates
data_history = deque(maxlen=20) # Store the last 20 data points
time_history = deque(maxlen=20) # Store timestamps

API_URL = "http://0.0.0.0:8000/predict"

def get_data_and_update_plot():
    try:
        r = requests.get(API_URL)
        data = r.json()
        price = data.get("result")

        if price is not None:
            now = datetime.datetime.now()
            data_history.append(price)
            time_history.append(now)

            # Create a Plotly figure
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=list(time_history), y=list(data_history), mode='lines+markers'))

            fig.update_layout(
                title="Real-time BTC/USD Price",
                xaxis_title="Time",
                yaxis_title="Price",
                xaxis=dict(tickformat="%H:%M:%S")
            )

            return fig, gr.update(value=f'{now:%H:%M:%S} → BTC/USD: {price}')
        else:
            return go.Figure(), gr.update(value=f'{datetime.datetime.now():%H:%M:%S} → Error: Could not get price')

    except requests.exceptions.RequestException as e:
        return go.Figure(), gr.update(value=f'{datetime.datetime.now():%H:%M:%S} → Error: {e}')
    except Exception as e:
        return go.Figure(), gr.update(value=f'{datetime.datetime.now():%H:%M:%S} → An unexpected error occurred: {e}')


with gr.Blocks() as app:
    gr.Markdown("### อัปเดตราคาค่าเงินจาก API และแสดงผลเป็นกราฟ")

    with gr.Row():
        plot_output = gr.Plot(label="Realtime Price Graph")
        text_output = gr.Textbox(label="Latest Update", lines=2, interactive=False)

    # Gradio 4.x+ syntax
    gr.Timer(value=1).tick(
        fn=get_data_and_update_plot,
        outputs=[plot_output, text_output]
    )

app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4003a1036954dba9ab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install pybullet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!git clone https://github.com/utiasDSL/gym-pybullet-drones.git

                ↑  Z  (ขึ้น)
                |
                |
                |         🟦 Drone (โดรน)
                |        /
                |       /
                |      /
                |     /
                |    /
                |   /
                |  /
                | /
                +--------------------→  X (ไปข้างหน้า)
               /
              /
             /
            /
           ↓
          Y (ไปทางขวา)


In [ ]:
with open('main.py', 'w') as f:
    f.write('''
import pybullet as p
import pybullet_data
import time
import datetime
import json
from fastapi import FastAPI
from fastapi.responses import JSONResponse
# import uvicorn
import threading

drone_state = {"x": 0, "y": 0, "z": 0, "vx": 0, "vy": 0, "vz": 0}

def simulate_drone():
    physicsClient = p.connect(p.DIRECT)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.81)
    p.loadURDF("plane.urdf")

    drone_path = "/content/gym-pybullet-drones/gym_pybullet_drones/assets/cf2x.urdf"
    drone = p.loadURDF(drone_path, [0, 0, 0])

    mass = p.getDynamicsInfo(drone, -1)[0]
    thrust = mass * 9.81 * 1.1  # ให้แรงมากกว่า hover เล็กน้อย

    while True:
        p.applyExternalForce(drone, -1, [0, 0, thrust], [0, 0, 0], p.WORLD_FRAME)
        p.stepSimulation()

        pos, _ = p.getBasePositionAndOrientation(drone)
        vel, _ = p.getBaseVelocity(drone)

        # คืนค่าเวลาปัจจุบันเป็นวินาทีตั้งแต่ 1 ม.ค. 1970
        now_epoch = time.time()
        now_iso = datetime.datetime.utcfromtimestamp(now_epoch).isoformat(timespec='microseconds') + "Z"

        drone_state.update({
            "x": pos[0],
            "y": pos[1],
            "z": pos[2],
            "vx": vel[0],
            "vy": vel[1],
            "vz": vel[2],
            "timestamp": now_iso,
        })

        time.sleep(1/240)

# --- รัน PyBullet ใน background ---
threading.Thread(target=simulate_drone, daemon=True).start()

# --- สร้าง API ---
app = FastAPI()

@app.get("/drone")
def get_drone_state():
    return JSONResponse(content=drone_state)

''')

In [ ]:
!kill -9 $(lsof -t -i:8000)

In [ ]:
!nohup uvicorn main:app --host 0.0.0.0 --port 8000 --reload &

In [ ]:
# !cat nohup.out

In [ ]:
!curl -X GET http://0.0.0.0:8000/drone

In [ ]:
import gradio as gr
import requests
import plotly.graph_objects as go
from collections import deque

API_URL = "http://0.0.0.0:8000/drone"

# เก็บประวัติพิกัด (200 จุดล่าสุด)
x_history, y_history, z_history = deque(maxlen=200), deque(maxlen=200), deque(maxlen=200)

def get_drone_data():
    try:
        r = requests.get(API_URL, timeout=1)
        data = r.json()

        x, y, z = data["x"], data["y"], data["z"]
        vx, vy, vz = data["vx"], data["vy"], data["vz"]

        speed_kmh = ((vx**2 + vy**2 + vz**2) ** 0.5) * 3.6

        x_history.append(x)
        y_history.append(y)
        z_history.append(z)

        pos_str = (
            f"Position (World Frame) "
            f"X={x:.2f} m, Y={y:.2f} m, Z={z:.2f} m\n\n"
            f"Velocity "
            f"vx={vx:.3f}, vy={vy:.3f}, vz={vz:.3f} m/s\n\n"
            f"Speed = {speed_kmh:.2f} km/h"
        )

        # --- สร้าง Figure ---
        fig3d = go.Figure()

        # path
        fig3d.add_trace(go.Scatter3d(
            x=list(x_history),
            y=list(y_history),
            z=list(z_history),
            mode="lines+markers",
            line=dict(color=list(range(len(x_history))), colorscale="Turbo", width=5),
            marker=dict(size=3, color=list(range(len(x_history))), colorscale="Turbo", showscale=False),
            name="Path"
        ))

        # current drone
        fig3d.add_trace(go.Scatter3d(
            x=[x], y=[y], z=[z],
            mode="markers",
            marker=dict(size=8, color="lime", symbol="circle"),
            name="Current Position"
        ))

        # --- จดจำมุมกล้องเดิมไว้ ---
        camera = fig3d.layout.scene.camera if "scene" in fig3d.layout else None

        margin = 5
        if len(x_history) > 1:
            x_range = [min(x_history)-margin, max(x_history)+margin]
            y_range = [min(y_history)-margin, max(y_history)+margin]
            z_range = [min(z_history)-margin, max(z_history)+margin]
        else:
            # กรอบเริ่มต้นกรณีข้อมูลยังน้อย
            x_range, y_range, z_range = [-10, 10], [-10, 10], [0, 20]

        # --- อัปเดต layout ---
        fig3d.update_layout(
            scene=dict(
                xaxis=dict(title="X (Forward, m)", range=x_range),
                yaxis=dict(title="Y (Right, m)", range=y_range),
                zaxis=dict(title="Z (Up, m)", range=z_range),
                aspectmode="cube"
            ),
            template="plotly_white",
            margin=dict(l=0, r=0, t=40, b=0),
            height=800
        )

        if camera:
            fig3d.update_layout(scene_camera=camera)

        return fig3d, pos_str

    except Exception as e:
        return go.Figure(), f"Error: {e}"


# --- สร้าง Gradio Dashboard ---
with gr.Blocks(
    css="""
        .gradio-container {max-width: 100% !important; padding: 0; margin: 0;}
        #plot-area {width: 100%; height: 85vh !important;} /* เต็มหน้าจอ */
        #header-text {text-align: center; margin-top: 8px;}
    """
) as dashboard:
    gr.Markdown("## 🚀 Drone Position & Velocity", elem_id="header-text")

    info = gr.Textbox(label="Drone Status", lines=6, interactive=False)
    plot3d = gr.Plot(label="🛰️ Real-time 3D Drone Monitor", elem_id="plot-area")

    gr.Timer(value=1).tick(fn=get_drone_data, outputs=[plot3d, info])

dashboard.launch()


In [ ]:
import requests
from collections import deque

API_URL = "http://0.0.0.0:8000/drone"

# เก็บประวัติพิกัด (x จุดล่าสุด)
window_size = 20
x_history, y_history, z_history, iso_history = deque(maxlen=window_size), deque(maxlen=window_size), deque(maxlen=window_size), deque(maxlen=window_size)

r = requests.get(API_URL, timeout=1)
data = r.json()
x, y, z, ios = data["x"], data["y"], data["z"], data["timestamp"]

print(x, y, z, ios)

x_history.append(x)
y_history.append(y)
z_history.append(z)
iso_history.append(ios)

x_history, y_history, z_history, iso_history

In [ ]:
import time
import datetime
from collections import deque
import requests
import numpy as np

history = deque(maxlen=window_size)

for _ in range(window_size):
    r = requests.get(API_URL, timeout=1)
    data = r.json()
    # หน่วยของ timestamp() คือ วินาที
    t = datetime.datetime.fromisoformat(data["timestamp"].replace("Z", "+00:00")).timestamp()
    x, y, z = data["x"], data["y"], data["z"]
    history.append((t, x, y, z))
    time.sleep(0.5)

                ↑  Z  (ขึ้น)
                |
                |
                |         🟦 Drone (โดรน)
                |        /
                |       /
                |      /
                |     /
                |    /
                |   /
                |  /
                | /
                +--------------------→  X (ไปข้างหน้า)
               /
              /
             /
            /
           ↓
          Y (ไปทางขวา)


In [ ]:
# ตำแหน่งป้อมปืน
Pg = np.array([0.0, 0.0, 0.0])

# ความเร็วของกระสุนปืน 200 m/s
Vs = 200.0

history[0][0]

In [ ]:
# ตำแหน่ง drone ปัจจุบัน

Pd = np.array(history[-1][1:4])
Pd

In [ ]:
times = np.array([h[0] for h in history])
coords = np.array([[h[1], h[2], h[3]] for h in history])

In [ ]:
times.shape, coords.shape

In [ ]:
t0 = times.mean()
t0

In [ ]:
t_rel = times - t0
t_rel

In [ ]:
Vd = np.zeros(3, dtype=float)
Vd

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=t_rel, y=coords[:, 2], mode='markers'))
fig.update_layout(title='Z Coordinate over time', xaxis_title='t_rel (seconds)', yaxis_title='Z Coordinate')
fig.show()

In [ ]:
for i in range(3):
  # fit line coord = a * t_rel + b
  # a คือ slope คือ ความเร็วในแกนนั้น
  # b คือ ค่าพิกัดของ Z ที่เวลาอ้างอิง (t_rel = 0)
  a, b = np.polyfit(t_rel, coords[:, i], 1)
  Vd[i] = a

Vd

In [ ]:
t_fit = np.linspace(t_rel.min(), t_rel.max(), 100)
y_fit_z = a * t_fit + b

fig = go.Figure()

fig.add_trace(go.Scatter(x=t_rel, y=coords[:, 2], mode='markers', name='Original Z Data'))

fig.add_trace(go.Scatter(x=t_fit, y=y_fit_z, mode='lines', name='Linear Fit (Z)', line=dict(color='red')))

fig.update_layout(title='Z Coordinate with Linear Fit', xaxis_title='t_rel (seconds)', yaxis_title='Z Coordinate')
fig.show()

In [ ]:
# ตำแหน่ง drone ปัจจุบัน
Pd

In [ ]:
# ความเร็วของโดรน
Vd

In [ ]:
# ตำแหน่งป้อมปืน
Pg

In [ ]:
# ความเร็วของกระสุน
Vs

In [ ]:
# ตำแหน่งของโดรน (Pd) เทียบกับตำแหน่งของป้อมปืน (Pg)
P_rel = Pd - Pg
P_rel

เราต้องหาค่าเวลา t ที่กระสุนกับโดรนจะอยู่ที่จุดเดียวกัน
at^2+bt+c=0

a บอกว่า กระสุนเร็วกว่าหรือช้ากว่าโดรนมากแค่ไหน

b บอกว่า โดรนกำลังเคลื่อนที่เข้าหรือออกจากป้อมเร็วแค่ไหน

c คือ โดรนอยู่ห่างจากป้อมแค่ไหนในตอนเริ่มต้น

In [ ]:
# vx² + vy² + vz²
a = np.dot(Vd, Vd) - Vs**2

a

In [ ]:
b = 2.0 * np.dot(P_rel, Vd)
b

In [ ]:
c = np.dot(P_rel, P_rel)
c

AI Projectile Intercept Formula for Gaming, without Trigonometry

https://medium.com/andys-coding-blog/ai-projectile-intercept-formula-for-gaming-without-trigonometry-37b70ef5718b


In [ ]:
# quadratic
disc = b*b - 4*a*c
sqrt_disc = np.sqrt(disc)
t1 = (-b + sqrt_disc) / (2*a)
t2 = (-b - sqrt_disc) / (2*a)

t1, t2

In [ ]:
ts = [t for t in (t1, t2) if t > 1e-9]
ts

In [ ]:
t = min(ts)
t

In [ ]:
P_hit = Pd + Vd * t
P_hit

In [ ]:
def estimate_velocity_linear_regression(history):
    if len(history) < 2:
        return np.array([0.0, 0.0, 0.0])
    times = np.array([h[0] for h in history])
    coords = np.array([[h[1], h[2], h[3]] for h in history])
    t0 = times.mean()
    t_rel = times - t0
    v = np.zeros(3, dtype=float)
    for i in range(3):
        a, b = np.polyfit(t_rel, coords[:, i], 1)
        v[i] = a
    return v

def compute_intercept_point(Pd, Vd, Pg, Vs, eps=1e-6):
    P_rel = Pd - Pg
    a = np.dot(Vd, Vd) - Vs**2
    b = 2.0 * np.dot(P_rel, Vd)
    c = np.dot(P_rel, P_rel)

    # if a is approx zero -> linear equation b t + c = 0
    if abs(a) < eps:
        if abs(b) < eps:
            return (None, None)  # no motion relative or degenerate
        t = -c / b
        if t > 0:
            P_hit = Pd + Vd * t
            return P_hit, t
        else:
            return (None, None)

    disc = b*b - 4*a*c
    if disc < 0:
        return (None, None)
    sqrt_disc = np.sqrt(disc)
    t1 = (-b + sqrt_disc) / (2*a)
    t2 = (-b - sqrt_disc) / (2*a)

    ts = [t for t in (t1, t2) if t > 1e-9]
    if not ts:
        return (None, None)
    t = min(ts)
    P_hit = Pd + Vd * t
    return P_hit, t

การคำนวนการหมุนปากลำกล้องปืน

In [ ]:
# ตำแหน่งป้อมปืน
Pg

In [ ]:
# ปากลำกล้องยื่นออก 0.5 m สูง 0.2 m
barrel_offset = np.array([0.5, 0, 0.2])
barrel_offset

In [ ]:
Pg_barrel = Pg + barrel_offset
Pg_barrel

In [ ]:
# จุดที่จะยิงโดย
P_hit

In [ ]:
# คำนวณเวกเตอร์จาก ปากลำกล้องไปจุดเป้า

D = P_hit - Pg_barrel
D

In [ ]:
dx, dy, dz = D

# มุมหมุนซ้าย/ขวา
yaw = np.arctan2(dy, dx)

# มุมเงย/ก้ม
# pitch = np.arctan2(dz, np.sqrt(dx**2 + dy**2))
pitch = np.arctan2(dz, np.hypot(dx, dy))

yaw_deg = np.degrees(yaw)
pitch_deg = np.degrees(pitch)

print(f"Yaw (radians) {yaw}, (degrees) {yaw_deg}")
print(f"Pitch (radians) {pitch}, (degrees) {pitch_deg}")

In [ ]:
P_hit, t = compute_intercept_point(Pd, Vd, Pg_barrel, Vs)
P_hit, t

In [ ]:
def compute_yaw_pitch(P_hit, Pg_barrel):
  dx, dy, dz = np.array(P_hit) - np.array(Pg_barrel)
  yaw = np.degrees(np.arctan2(dy, dx))
  pitch = np.degrees(np.arctan2(dz, np.hypot(dx, dy)))
  return yaw, pitch

In [ ]:
yaw, pitch = compute_yaw_pitch(P_hit, Pg_barrel)
yaw, pitch

Code เต็ม

In [ ]:
import gradio as gr
import requests
import plotly.graph_objects as go
from collections import deque
import numpy as np
import datetime
import time

API_URL = "http://0.0.0.0:8000/drone"

# เก็บประวัติพิกัด/เวลา (สำหรับ estimate velocity)
window_size = 20
history = deque(maxlen=window_size)

# เก็บประวัติแค่เพื่อวาด path
x_history, y_history, z_history = deque(maxlen=200), deque(maxlen=200), deque(maxlen=200)

# ป้อมปืน / barrel offset (ปรับตามของจริง)
Pg = np.array([0.0, 0.0, 0.0])
barrel_offset = np.array([0.5, 0.0, 0.2])   # ตัวอย่าง ปากลำกล้องยื่นไป 0.5m และสูง 0.2m
Pg_barrel = Pg + barrel_offset

# ความเร็วกระสุน (m/s)
Vs = 200.0

firing = False               # True เมื่อ user กด Fire และยังไม่ถึงเวลา impact
fire_target_epoch = None     # epoch (seconds) เมื่อควรยิงจริง (now + t)
last_P_hit = None            # เก็บ P_hit ปัจจุบัน (สำหรับแสดง marker impact)
impact_record = None         # เก็บข้อมูลตอน fired (P_hit, epoch)

def estimate_velocity_linear_regression(history):
    if len(history) < 2:
        return np.array([0.0, 0.0, 0.0])
    times = np.array([h[0] for h in history])
    coords = np.array([[h[1], h[2], h[3]] for h in history])
    t0 = times.mean()
    t_rel = times - t0
    v = np.zeros(3, dtype=float)
    for i in range(3):
        a, b = np.polyfit(t_rel, coords[:, i], 1)  # a = slope = velocity
        v[i] = a
    return v

def compute_intercept_point(Pd, Vd, Pg, Vs, eps=1e-6):
    P_rel = Pd - Pg
    a = np.dot(Vd, Vd) - Vs**2
    b = 2.0 * np.dot(P_rel, Vd)
    c = np.dot(P_rel, P_rel)

    if abs(a) < eps:
        # linear: b t + c = 0
        if abs(b) < eps:
            return (None, None)
        t = -c / b
        if t > 0:
            P_hit = Pd + Vd * t
            return P_hit, t
        else:
            return (None, None)

    disc = b*b - 4*a*c
    if disc < 0:
        return (None, None)
    sqrt_disc = np.sqrt(disc)
    t1 = (-b + sqrt_disc) / (2*a)
    t2 = (-b - sqrt_disc) / (2*a)
    ts = [t for t in (t1, t2) if t > 1e-9]
    if not ts:
        return (None, None)
    t = min(ts)
    P_hit = Pd + Vd * t
    return P_hit, t

def compute_yaw_pitch(P_hit, Pg_barrel):
    dx, dy, dz = np.array(P_hit) - np.array(Pg_barrel)
    yaw = np.degrees(np.arctan2(dy, dx))
    pitch = np.degrees(np.arctan2(dz, np.hypot(dx, dy)))
    return float(yaw), float(pitch)

def prepare_fire():
    """
    เมื่อกด Fire: อ่านสถานะปัจจุบันของโดรน (ล่าสุดจาก history if available),
    คำนวณ P_hit,t แล้วตั้ง fire_target_epoch = now + t
    """
    global firing, fire_target_epoch, last_P_hit, impact_record

    # ต้องมีข้อมูลล่าสุดใน history
    if len(history) == 0:
        return "No drone data to compute intercept (history empty)."

    # ใช้สถานะปัจจุบัน (ล่าสุด)
    t_now, x, y, z = history[-1]
    Pd = np.array([x, y, z])
    Vd = estimate_velocity_linear_regression(history)
    P_hit, t_to_hit = compute_intercept_point(Pd, Vd, Pg_barrel, Vs)

    if P_hit is None or t_to_hit is None:
        return "No intercept solution (cannot fire)."

    last_P_hit = P_hit
    firing = True
    fire_target_epoch = time.time() + float(t_to_hit)
    impact_record = None
    fire_time_iso = datetime.datetime.utcfromtimestamp(fire_target_epoch).isoformat(timespec='seconds') + "Z"
    return f"Firing prepared — impact in {t_to_hit:.2f} s (target at {fire_time_iso} UTC)."

def get_drone_data_and_plot():
    global last_P_hit, firing, fire_target_epoch, impact_record

    try:
        r = requests.get(API_URL, timeout=1)
        data = r.json()

        # position & velocity from API
        x, y, z = data["x"], data["y"], data["z"]
        vx, vy, vz = data["vx"], data["vy"], data["vz"]

        # timestamp from API (ISO) -> seconds
        ts_str = data.get("timestamp")
        if ts_str:
            t_seconds = datetime.datetime.fromisoformat(ts_str.replace("Z", "+00:00")).timestamp()
        else:
            t_seconds = time.time()

        # update histories
        history.append((t_seconds, x, y, z))
        x_history.append(x); y_history.append(y); z_history.append(z)

        # estimate velocity from recent history
        Vd = estimate_velocity_linear_regression(history)

        # current drone pos
        Pd = np.array([x, y, z])

        # compute intercept (update aiming)
        P_hit, t_to_hit = compute_intercept_point(Pd, Vd, Pg_barrel, Vs)
        if P_hit is not None:
            last_P_hit = P_hit

        # prepare status string (pos, vel, speed)
        speed_kmh = ((vx**2 + vy**2 + vz**2) ** 0.5) * 3.6
        pos_str = (
            f"Position (World Frame)\n"
            f"({x:.2f}, {y:.2f}, {z:.2f}) m\n\n"
            f"Velocity (API)\n"
            f"({vx:.3f}, {vy:.3f}, {vz:.3f}) m/s\n\n"
            f"Speed\n{speed_kmh:.2f} km/h\n\n"
        )

        if P_hit is not None and t_to_hit is not None:
            yaw_deg, pitch_deg = compute_yaw_pitch(P_hit, Pg_barrel)
            pos_str += f"<<Current predicted HIT>>\nt\n{t_to_hit:.2f}s\n\nP_hit\n({P_hit[0]:.3f}, {P_hit[1]:.3f}, {P_hit[2]:.3f}) m\n\n"
            pos_str += f"Yaw\n({yaw_deg:.2f}°, Pitch={pitch_deg:.2f}°)\n"
        else:
            pos_str += "No intercept solution (P_hit not found)\n"

        # If firing was prepared, compute countdown to target epoch
        if firing and fire_target_epoch is not None:
            remaining = fire_target_epoch - time.time()
            if remaining > 0:
                pos_str += f"\nFIRING -> countdown: {remaining:.2f} s\n"
            else:
                # impact moment
                pos_str += f"\n*** IMPACT / FIRED at {datetime.datetime.utcnow().isoformat(timespec='seconds')}Z ***\n"
                firing = False
                impact_record = {
                    "epoch": fire_target_epoch,
                    "P_hit": last_P_hit.copy() if last_P_hit is not None else None
                }
                fire_target_epoch = None

        if impact_record is not None:
            ph = impact_record["P_hit"]
            if ph is not None:
                pos_str += f"\nLast Impact at P_hit = ({ph[0]:.3f}, {ph[1]:.3f}, {ph[2]:.3f}) m\n"

        fig3d = go.Figure()

        # path
        fig3d.add_trace(go.Scatter3d(
            x=list(x_history),
            y=list(y_history),
            z=list(z_history),
            mode="lines+markers",
            line=dict(color=list(range(len(x_history))), colorscale="Turbo", width=4),
            marker=dict(size=3, color=list(range(len(x_history))), colorscale="Turbo", showscale=False),
            name="Path"
        ))

        # current drone marker
        fig3d.add_trace(go.Scatter3d(
            x=[x], y=[y], z=[z],
            mode="markers",
            marker=dict(size=8, color="lime", symbol="circle"),
            name="Drone (current)"
        ))

        # barrel position marker
        fig3d.add_trace(go.Scatter3d(
            x=[Pg_barrel[0]], y=[Pg_barrel[1]], z=[Pg_barrel[2]],
            mode="markers+text",
            marker=dict(size=6, color="black", symbol="square"),
            text=["Barrel"],
            textposition="top center",
            name="Barrel"
        ))

        # P_hit marker & line (current predicted)
        if last_P_hit is not None:
            fig3d.add_trace(go.Scatter3d(
                x=[last_P_hit[0]], y=[last_P_hit[1]], z=[last_P_hit[2]],
                mode="markers+text",
                marker=dict(size=6, color="red", symbol="x"),
                text=[f"P_hit\n{last_P_hit[2]:.1f}m"],
                textposition="bottom center",
                name="P_hit (pred)"
            ))
            # line from barrel -> P_hit
            fig3d.add_trace(go.Scatter3d(
                x=[Pg_barrel[0], last_P_hit[0]],
                y=[Pg_barrel[1], last_P_hit[1]],
                z=[Pg_barrel[2], last_P_hit[2]],
                mode="lines",
                line=dict(color="red", width=3, dash="dash"),
                showlegend=False
            ))

        # impact marker (if fired)
        if impact_record is not None and impact_record.get("P_hit") is not None:
            ph = impact_record["P_hit"]
            fig3d.add_trace(go.Scatter3d(
                x=[ph[0]], y=[ph[1]], z=[ph[2]],
                mode="markers+text",
                marker=dict(size=10, color="orange", symbol="diamond"),
                text=["IMPACT"],
                textposition="top center",
                name="Impact"
            ))

        margin = 5.0
        if len(x_history) > 1:
            x_range = [min(x_history)-margin, max(x_history)+margin]
            y_range = [min(y_history)-margin, max(y_history)+margin]
            z_range = [min(z_history)-margin, max(z_history)+margin]
        else:
            x_range, y_range, z_range = [-10,10], [-10,10], [0,20]

        fig3d.update_layout(
            scene=dict(
                xaxis=dict(title="X (Forward, m)", range=x_range),
                yaxis=dict(title="Y (Right, m)", range=y_range),
                zaxis=dict(title="Z (Up, m)", range=z_range),
                aspectmode="cube"
            ),
            template="plotly_white",
            margin=dict(l=0, r=0, t=40, b=0),
            height=800
        )

        return fig3d, pos_str

    except Exception as e:
        return go.Figure(), f"Error: {e}"

# สร้าง Gradio Dashboard
with gr.Blocks(
    css="""
        .gradio-container {max-width: 100% !important; padding: 0; margin: 0;}
        #plot-area {width: 100%; height: 75vh !important;} /* ปรับสูงให้พอดี */
        #header-text {text-align: center; margin-top: 8px;}
        .controls {display:flex; gap:8px; align-items:center;}
    """
) as dashboard:
    gr.Markdown("## 🚀 Drone Position, Predicted Hit & Gun Angles (Fire)", elem_id="header-text")

    with gr.Row():
        with gr.Column(scale=1):
            btn_fire = gr.Button("Fire (Prepare)")
            info = gr.Textbox(label="Drone Status & Intercept Info", lines=10, interactive=False)
        with gr.Column(scale=3):
            plot3d = gr.Plot(label="🛰️ Real-time 3D Drone Monitor", elem_id="plot-area")

    # ปุ่ม callback
    btn_fire.click(prepare_fire, outputs=info)

    # เรียกทุก 1 วินาที (update กราฟ และสถานะ)
    gr.Timer(value=1).tick(fn=get_drone_data_and_plot, outputs=[plot3d, info])

dashboard.launch()
